## MAX CUT SDP relaxation
### Graph based on viewers rating movies
Here we solve the relaxed SDP to find the MAX cut in a graph. The goal is to find integral solution for $\gamma$-stable graphs

In [ ]:
%matplotlib inline
import networkx as nx
import numpy as np
import pylab

from algo import build_laplacian, solve_sdp, assignment_solution_sdp
from data_proc import build_a, build_graph, build_user_to_movies

## Fake data to illustrate the integral solution
Easy max cut solution. 

In [ ]:
W = np.array([[0,4,0,8],[4,0,0,2],[0,0,0,7],[8,2,7,0]])
L = build_laplacian(W)
X = solve_sdp(L, triangle_inequalities=True)

In [ ]:
# assignment
X_array=np.array(X)
assignment_X = assignment_solution_sdp(X_array)

In [ ]:
#display the graph
G = nx.from_numpy_matrix(W)
fig=pylab.figure(figsize=(11,8))

colors = ['g', 'r']

node_colors=['w']*W.shape[0]
for node, cluster in enumerate(assignment_X):
    node_colors[node] = colors[int(cluster)]

pos=nx.spring_layout(G)
#edges
nx.draw_networkx(G,pos,
                edgelist=[e for e in G.edges()],
                node_color=node_colors)


labels={e:'{0}'.format(W[e]) for e in G.edges()}
#flow label
nx.draw_networkx_edge_labels(G, pos,
                        edge_labels=labels)

#hide axis
fig.gca().axes.get_xaxis().set_ticks([])
fig.gca().axes.get_yaxis().set_ticks([])

pylab.show()

#### We introduce a gamma perturbation for all edges

In [ ]:
gamma = 2
W_gamma1 = gamma * W
L = build_laplacian(W_gamma1)
X = solve_sdp(L, triangle_inequalities=True)
print(X)
X_array=np.array(X)
assignment_X = assignment_solution_sdp(X_array)
print(assignment_X)

#### We introduce a gamma perturbation for only part of the edges

In [ ]:
gamma = 2 # if gamma = 1000 it may not work depending of the mask
# random boolean mask for which values will be changed
mask = np.random.randint(0,2,size=W.shape).astype(np.bool)

# random matrix the same shape of the data
W = np.array([[0,4,0,8],[4,0,0,2],[0,0,0,7],[8,2,7,0]])
r = gamma * W

# use your mask to replace values in your input array
W_gamma2 = W
W_gamma2[mask] = r[mask]
L = build_laplacian(W)
X = solve_sdp(L, triangle_inequalities=True)
print(X)
X_array=np.array(X)
assignment_X = assignment_solution_sdp(X_array)
print(assignment_X)

## Simulated data
Simulating a bipartite graph with viewers giving grades (between -2.5 and 2.5) to movies.

In [ ]:
W = np.array([[0,0,0,-2,0,-0.5],[0,0,0,2,2.5,0],[0,0,0,0,-1,2.5],[-2,2,0,0,0,0],[0,2.5,-1,0,0,0],[-0.5,0,2.5,0,0,0]])
L = build_laplacian(W)
X = solve_sdp(L, triangle_inequalities=True)
X_array=np.array(X)
assignment_X = assignment_solution_sdp(X_array)

In [ ]:
#display the graph
G = nx.from_numpy_matrix(W)
fig=pylab.figure(figsize=(11,8))

colors = ['g', 'r']

node_colors=['w']*W.shape[0]
for node, cluster in enumerate(assignment_X):
    node_colors[node] = colors[int(cluster)]

pos=nx.spring_layout(G)
#edges
nx.draw_networkx(G,pos,
                edgelist=[e for e in G.edges()],
                node_color=node_colors)


labels={e:'{0}'.format(W[e]) for e in G.edges()}
#flow label
nx.draw_networkx_edge_labels(G, pos,
                        edge_labels=labels)

#hide axis
fig.gca().axes.get_xaxis().set_ticks([])
fig.gca().axes.get_yaxis().set_ticks([])

pylab.show()

#### We introduce a gamma perturbation for all edges

In [ ]:
gamma = 2
W_gamma1 = gamma * W
L = build_laplacian(W_gamma1)
X = solve_sdp(L, triangle_inequalities=True)
print(X)
X_array=np.array(X)
assignement_X = assignment_solution(X_array)
print(assignement_X)

#### We introduce a gamma perturbation for only one edge

In [ ]:
gamma = 10 # if gamma = 1000 it may not work anymore
# random boolean mask for which values will be changed
W_gamma2 = np.array([[0,0,0,-2,0,-0.5*gamma],[0,0,0,2,2.5,0],[0,0,0,0,-1,2.5],[-2,2,0,0,0,0],[0,2.5,-1,0,0,0],[-0.5*gamma,0,2.5,0,0,0]])

In [ ]:
L = build_laplacian(W_gamma2)
X = solve_sdp(L, triangle_inequalities=True)
print(X)
X_array=np.array(X)
assignement_X = assignment_solution(X_array)
print(assignement_X)

## Real data from themoviedb.org
Extracting  a sub graph of the entire database

In [ ]:
# building the summary dictionary
summary_dictionary = build_user_to_movies('movielens.tsv')
# unpacking of the dictionary
users_to_movies = summary_dictionary['users_to_movies']
n_users = summary_dictionary['n_users']
k_users = 4
n_movies = summary_dictionary['n_movies']
k_movies = 4
# building the rating matrix
a = build_a(n_users, k_users, n_movies, k_movies, users_to_movies)
# building the adjacency matrix
W = build_graph(k_users, k_movies, a)
print(W)

In [ ]:
# the laplacian of the graph
L = build_laplacian(W)
X = solve_sdp(L, triangle_inequalities=True)
X_array=np.array(X)
assignment_X = assignment_solution_sdp(X_array)

In [ ]:
#display the graph
G = nx.from_numpy_matrix(W)
fig=pylab.figure(figsize=(11,8))

colors = ['g', 'r']

node_colors=['w']*W.shape[0]
for node, cluster in enumerate(assignment_X):
    node_colors[node] = colors[int(cluster)]

pos=nx.spring_layout(G)
#edges
nx.draw_networkx(G,pos,
                edgelist=[e for e in G.edges()],
                node_color=node_colors)


labels={e:'{0}'.format(W[e]) for e in G.edges()}
#flow label
nx.draw_networkx_edge_labels(G, pos,
                        edge_labels=labels)

#hide axis
fig.gca().axes.get_xaxis().set_ticks([])
fig.gca().axes.get_yaxis().set_ticks([])

pylab.show()

#### We introduce a gamma perturbation for only part of the edges

In [ ]:
gamma = 2 
# random boolean mask for which values will be changed
mask = np.random.randint(0,2,size=W.shape).astype(np.bool)

# random matrix the same shape of the data
r = gamma * W

# use your mask to replace values in your input array
W_gamma2 = W.copy()
W_gamma2[mask] = r[mask]
L = build_laplacian(W_gamma2)
X = solve_sdp(L, triangle_inequalities=True)
print(X)
X_array=np.array(X)
assignement_X = assignment_solution(X_array)
print(assignement_X)